In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import shap
import whois
from __future__ import print_function
import pyzbar.pyzbar as pyzbar
import cv2
import re
import time
from sklearn import tree
import pydotplus as pdp
import matplotlib.pyplot as plt
import matplotlib.image as pltimg

def qrcoderead():

    cap = cv2.VideoCapture(0)

    cap.set(3,640)
    cap.set(4,480)
    #160.0 x 120.0
    #176.0 x 144.0
    #320.0 x 240.0
    #352.0 x 288.0
    #640.0 x 480.0
    #1024.0 x 768.0
    #1280.0 x 1024.0
    time.sleep(2)

    def decode(im) : 
        # Find barcodes and QR codes
        decodedObjects = pyzbar.decode(im) 
        return decodedObjects


    font = cv2.FONT_HERSHEY_SIMPLEX

    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()
        # Our operations on the frame come here
        im = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        decodedObjects = decode(im)

        for decodedObject in decodedObjects: 
            points = decodedObject.polygon

            # If the points do not form a quad, find convex hull
            if len(points) > 4 : 
              hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
              hull = list(map(tuple, np.squeeze(hull)))
            else : 
              hull = points;

            # Number of points in the convex hull
            n = len(hull)     
            # Draw the convext hull
            for j in range(0,n):
              cv2.line(frame, hull[j], hull[ (j+1) % n], (255,0,0), 3)

            x = decodedObject.rect.left
            y = decodedObject.rect.top
            barCode = str(decodedObject.data)
            cv2.putText(frame, barCode, (x, y), font, 1, (0,255,255), 2, cv2.LINE_AA)

        # Display the resulting frame
        cv2.imshow('frame',frame)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break
        elif key & 0xFF == ord('s'): # wait for 's' key to save 
            cv2.imwrite('Capture.png', frame)     

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    return decodedObject.data

#Above function was mostly obtained from "https://github.com/cuicaihao/Webcam_QR_Detector". Alterations were made for this poject but the general code is still the same.

dftest = pd.read_csv(r"C:\Users\bulle_000\Desktop\send help\dataset.csv")
dftest = dftest.head(1)
z = ["URLURL_Length", "Shortining_Service", "having_At_Symbol", "double_slash_redirecting", "Prefix_Suffix", "SSLfinal_State", "HTTPS_token", "web_traffic" ,"age_of_domain", "Domain_registeration_length", "having_IPhaving_IP_Address", "Abnormal_URL", "URL_of_Anchor"]
dftest = dftest[z]

def test_len(x):
    if len(x) < 54:
        dftest.at[0,"URLURL_Length"] = 1
    elif len(x) < 76:
        dftest.at[0,"URLURL_Length"] = 0
    else:
        dftest.at[0,"URLURL_Length"] = -1
        

def url_short(x):
    if "bit.ly" in x:
        dftest.at[0,"Shortining_Service"] = -1
    elif "tinyurl.com" in x:
        dftest.at[0,"Shortining_Service"] = -1
    else:
        dftest.at[0,"Shortining_Service"] = 1

def has_at(x):
    if "@" in x:
        dftest.at[0,"having_At_Symbol"] = -1
    else:
        dftest.at[0,"having_At_Symbol"] = 1

def redirect(x):
    y = x[7:]
    if "//" in y:
        dftest.at[0,"double_slash_redirecting"] = -1
    else:
        dftest.at[0,"double_slash_redirecting"] = 1

def prefix(x):
    if "-" in x:
        dftest.at[0,"Prefix_Suffix"] = -1
    else:
        dftest.at[0,"Prefix_Suffix"] = 1

def https(x):
    if "https" in x[:10]:
        dftest.at[0,"SSLfinal_State"] = 1
    elif "http" in x[:10]: 
        dftest.at[0,"SSLfinal_State"] = -1
    else:
        dftest.at[0,"SSLfinal_State"] = 0

def httpstoken(x):
    if "https" in x[5:]:
        dftest.at[0,"HTTPS_token"] = -1
    else:
        dftest.at[0,"HTTPS_token"] = 1

def get_rank(x):
    import requests
    import sys
    from bs4 import BeautifulSoup
    import re
    import validators


    alexa_base_url = 'https://alexa.com/siteinfo/'
    site_name = x


    def is_valid_domain(site_name):
        if validators.domain(site_name):
            return True
        else:
            return False

    url_for_rank = alexa_base_url + site_name

    # Request formatted url for rank(s)
    page = requests.get(url_for_rank)
    soup = BeautifulSoup(page.content, 'html.parser')

    # get ranks text in a list
    country_ranks = soup.find_all('div', id='CountryRank')

    # select the data with class='rank-global' and the class='data'
    global_rank = soup.select('.rank-global .data')

    # Display Global rank safely
    try:
        match = re.search(r'[\d,]+', global_rank[0].text.strip())
    except:
        return "none"
    return match.group()    

def global_rank(x):
    y = get_rank(x)
    if y == "none":
        dftest.at[0,"web_traffic"] = -1
        return
    y = re.sub("[^0-9]", "",str(y))
    if int(y) < 100000:
        dftest.at[0,"web_traffic"] = 1
    else:
        dftest.at[0,"web_traffic"] = 0

def is_registered(domain_name):
    try:
        w = whois.whois(domain_name)
    except Exception:
        return False
    else:
        return bool(w.domain_name)
def domain_age(x):
    import re
    z = 0
    from datetime import date
    today = date.today()
    if is_registered(x):
        y = whois_info.creation_date
        y = re.sub("[^0-9]", "",str(y))
        y = y[:6]
        time = re.sub("[^0-9]", "", str(today))
        time = time[:6]
        if y == "":
            return z
        tpass = int(time) - int(y)
    else: 
        return z
    return tpass
def domainpostive(x):
    tp = domain_age(x)
    if tp > 5:
        dftest.at[0,"age_of_domain"] = 1
    else:
        dftest.at[0,"age_of_domain"] = -1
        
def reg_len(x):
    z=0
    from datetime import date
    today = date.today()
    if is_registered(x):
        y = whois_info.expiration_date
        y = re.sub("[^0-9]", "",str(y))
        y = y[:6]
        time = re.sub("[^0-9]", "", str(today))
        time = time[:6]
        if y == "":
            return z
        time_till = int(y) - int(time)
    else:
        return z
    return time_till

def reg_len_change(x):
    y = reg_len(x)
    if y < 100:
        dftest.at[0,"Domain_registeration_length"] = -1
    else: 
        dftest.at[0,"Domain_registeration_length"] = 1

def ipad(x):

    x = x[7:15]
    z = re.sub("[^0-9]", "", x)
    if len(z) > 5:
        dftest.at[0,"having_IPhaving_IP_Address"] = -1
    else: 
        dftest.at[0,"having_IPhaving_IP_Address"] = 1

def abnormal(x):
    y = "dfajksjdlasdl;ajsd;l"
    if is_registered(x):
        y = str(whois_info.domain_name)
        y = y.lower()
    if y in x:
        dftest.at[0,"Abnormal_URL"] = 1
    else:
        dftest.at[0,"Abnormal_URL"] = -1

def url_anchor(x):
    try:
        if is_registered(x):        
            domain = whois_info.domain_name
        response = requests.get(x)
        soup = BeautifulSoup(response.text, "html.parser")
        liss = list()
        count = 0
        countbad = 0
        for link in soup.findAll('a'):
            liss.append(link.get('href'))
        for i in range(len(liss)):
            string = liss[i]
            try:
                if "http" in string:
                    if x in string:
                        count = count
                    elif domain in string:
                        count = count
                    else:
                        countbad += 1
            except:
                count -= 1
            count += 1
        if count == 0:
            dftest.at[0, "URL_of_Anchor"] = 0
            return 1
        returnvalue = countbad/count
        if returnvalue > 0.67:
            dftest.at[0, "URL_of_Anchor"] = -1
        elif returnvalue > 0.31:
             dftest.at[0, "URL_of_Anchor"] = 0
        else:
             dftest.at[0, "URL_of_Anchor"] = 1
    except:
        dftest.at[0, "URL_of_Anchor"] = 0

df = pd.read_csv(r"C:\Users\bulle_000\Desktop\send help\finaldata.csv")
df = df[:2000] 
df = df.sample(frac=1).reset_index(drop=True)



z = ["URLURL_Length", "Shortining_Service", "having_At_Symbol", "double_slash_redirecting", "Prefix_Suffix", "SSLfinal_State", "HTTPS_token", "web_traffic" ,"age_of_domain", "Domain_registeration_length", "having_IPhaving_IP_Address", "Abnormal_URL", "URL_of_Anchor"]
X = df[z] 
y = df["Label"]
X_test = X[1750:]
y_test = y[1750:]
X_train = X[:1750]
y_train = y[:1750]

dtree = RandomForestRegressor(n_estimators=20, random_state=0)
dtree = dtree.fit(X_train, y_train)

from sklearn import metrics
y_pred = dtree.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

def testqr(x):
    if is_registered(x):    
        whois_info = whois.whois(x)
    test_len(x)
    url_short(x)
    has_at(x)
    redirect(x)
    prefix(x)
    https(x)
    httpstoken(x)
    domainpostive(x)
    global_rank(x)
    reg_len_change(x)
    ipad(x)
    abnormal(x)
    url_anchor(x)
    y = dtree.predict(dftest)
    if y <-0.5:
        print("Probobly Phising, The closer to -1 the higher the chance!")
    elif y < 0:
        print("Possibly Phising, The closer to -1 the higher the chance!")
    elif y < 0.5:
        print("Probobly not, but still possibly Phising, the closer to 1 the safer you are!")
    else:
        print("Almost defenitely not Phising, the closer to 1 the safer you are!")
    print (y)

def main():
    x = qrcoderead()
    x = str(x)
    x = x[2:]
    x = x[:-1]
    if is_registered(x):    
        global whois_info 
        whois_info = whois.whois(x)
    test_len(x)
    url_short(x)
    has_at(x)
    redirect(x)
    prefix(x)
    https(x)
    httpstoken(x)
    domainpostive(x)
    global_rank(x)
    reg_len_change(x)
    ipad(x)
    abnormal(x)
    url_anchor(x)
    y = dtree.predict(dftest)
    if y <-5:
        print("Probobly Phising, The closer to -1 the higher the chance!")
    elif y < 0:
        print("Possibly Phising, The closer to -1 the higher the chance!")
    elif y < 0.5:
        print("Probobly not, but still possibly Phising, the closer to 1 the safer you are!")
    else:
        print("Almost defenitely not Phising, the closer to 1 the safer you are!")
    print (y)

if __name__ == "__main__":
    %%time
    main()